# 这是2024/03/03 对CSDN中python中pandas的复习

## Data School‘s top 25 pandas tricks


    df = df.rename({'col one':'col_one', 'col two':'col_two'}, axis='columns')
    df.columns = df.columns.str.replace(' ', '_')  将前者替换成后者
    
    对数据框的列名添加前后缀
    df.add_prefix('X_')
    df.add_suffix('_Y')
    
    Let's say you need to select only the numeric columns. You can use the select_dtypes() method:
    drinks.select_dtypes(include='number').head()
    drinks.select_dtypes(include='object').head()
    drinks.select_dtypes(include=['number', 'object', 'category', 'datetime']).head()
    drinks.select_dtypes(exclude='number').head()
    
    data中数据类型转换
    df.astype({'col_one':'float', 'col_two':'float'}).dtypes
    
    from glob import glob
    stock_files = sorted(glob('data/stocks*.csv'))
    stock_files  返回一个列表
    ['data/stocks1.csv', 'data/stocks2.csv', 'data/stocks3.csv']
    重点
    pd.concat((pd.read_csv(file) for file in stock_files))
    Unfortunately, there are now duplicate values in the index. 
    pd.concat((pd.read_csv(file) for file in stock_files), ignore_index=True)
    
    Split a DataFrame into two random subsets
    随机抽取data的subdata
    movies_1 = movies.sample(frac=0.75, random_state=1234)
    movies_2 = movies.drop(movies_1.index)
    movies_1 和movies_2的 “和” 即为 movies
    
    movies[movies.genre.isin(['Action', 'Drama', 'Western'])].head() *
    movies[(movies.genre == 'Action') |
           (movies.genre == 'Drama') |
           (movies.genre == 'Western')].head()
    上面两种方法效果是一样的
    取上面 * 的对立面
    movies[~movies.genre.isin(['Action', 'Drama', 'Western'])].head()
    
    Split a string into multiple columns  将一列拆分多列
    df[['first', 'middle', 'last']] = df.name.str.split(' ', expand=True)
    df['city'] = df.location.str.split(', ', expand=True)[0]
    
    Expand a Series of lists into a DataFrame  将一个list的列分为单个的列
    df = pd.DataFrame({'col_one':['a', 'b', 'c'], 'col_two':[[10, 40], [20, 50], [30, 60]]})
    df_new = df.col_two.apply(pd.Series)
    pd.concat([df, df_new], axis='columns')

    Combine the output of an aggregation with a DataFrame  增加一列的方式
    total_price = orders.groupby('order_id').item_price.transform('sum')
    orders['total_price'] = total_price
    orders['percent_of_total'] = orders.item_price / orders.total_price

    titanic.describe().loc['min':'max', 'Pclass':'Parch']

    titanic.groupby(['Sex', 'Pclass']).Survived.mean()
    titanic.groupby(['Sex', 'Pclass']).Survived.mean().unstack()

    22. Create a pivot table
    titanic.pivot_table(index='Sex', columns='Pclass', values='Survived', aggfunc='mean')
    titanic.pivot_table(index='Sex', columns='Pclass', values='Survived', aggfunc='count',margins=True)

    23. Convert continuous data into categorical data 将连续值转为分段变量
    pd.cut(titanic.Age, bins=[0, 18, 25, 99], labels=['child', 'young adult', 'adult']).head(10)

    24. Change display options
    pd.set_option('display.float_format', '{:.2f}'.format)

    25. Style a DataFrame
    format_dict = {'Date':'{:%m/%d/%y}', 'Close':'${:.2f}', 'Volume':'{:,}'}
    stocks.style.format(format_dict)
    (stocks.style.format(format_dict)
     .hide_index()
     .highlight_min('Close', color='red')
     .highlight_max('Close', color='lightgreen')
    )
    (stocks.style.format(format_dict)
     .hide_index()
     .background_gradient(subset='Volume', cmap='Blues')
    )


## pandas 个人不熟


    # use the '|' operator to specify that a row can match any of the three criteria
    movies[(movies.genre == 'Crime') | (movies.genre == 'Drama') | (movies.genre == 'Action')].head(10)

    # or equivalently, use the 'isin' method
    movies[movies.genre.isin(['Crime', 'Drama', 'Action'])].head(10)

    # specify how many rows to read
    ufo = pd.read_csv('data/ufo.csv',nrows=4)  # 取了前三行
    
    # 删除数据框中不是数值型的object

    drinks = pd.read_csv('data/drinks.csv')
    # only include numeric columns in the DataFrame
    import numpy as np
    drinks.select_dtypes(include=[np.number]).dtypes

    # 选择数据框中要describe的列
    # pass the string 'all' to describe all columns
    drinks.describe(include='all')
    # # pass a list of data types to only describe certain types
    drinks.describe(include=['object', 'float64'])
    # pass a list even if you only want to describe a single data type
    drinks.describe(include=['object'])

    # use the boolean Series to filter the DataFrame
    orders[orders.item_name.str.contains('Chicken')].head()
    ## orders.item_name.str.contains('Chicken')返回一系列bool值

    # string methods can be chained together
    orders.choice_description.str.replace('[', '').str.replace(']', '').head()

    #支持正则表达式
    # many pandas string methods support regular expressions (regex)
    orders.choice_description.str.replace('[\[\]]', '').head()

    # convert a boolean Series to an integer (False = 0, True = 1)
    orders.item_name.str.contains('Chicken').astype(int).value_counts()  
    # 注意 .value_counts 与 .value_counts()的区别

    # other aggregation functions (such as 'max') can also be used with groupby
    drinks.groupby('continent').beer_servings.max()
    # multiple aggregation functions can be applied simultaneously
    drinks.groupby('continent').beer_servings.agg(['count', 'mean', 'min', 'max'])

    NA 值
    # if 'any' values are missing in a row, then drop that row
    ufo.dropna(how='any').shape  # 只要含有Nan的值就drop
    # if 'all' values are missing in a row, then drop that row (none are dropped in this case)
    ufo.dropna(how='all').shape
    # if 'any' values are missing in a row (considering only 'City' and 'Shape Reported'), then drop that row
    ufo.dropna(subset=['City', 'Shape Reported'], how='any').shape
    # if 'all' values are missing in a row (considering only 'City' and 'Shape Reported'), then drop that row
    ufo.dropna(subset=['City', 'Shape Reported'], how='all').shape
    # 'value_counts' does not include missing values by default
    ufo['Shape Reported'].value_counts().head()
    # explicitly include missing values
    ufo['Shape Reported'].value_counts(dropna=False).head()  #  将 NaN算一个object进行统计
    # fill in missing values with a specified value
    ufo['Shape Reported'].fillna(value='VARIOUS', inplace=True)
    # confirm that the missing values were filled in
    ufo['Shape Reported'].value_counts().head()

    drinks.loc[23, 'beer_servings']
    drinks.set_index('country', inplace=True)
    drinks.index.name = None   # 设置index的名字
    drinks.reset_index(inplace=True)  # 还原index 
    # you can interact with any DataFrame using its index and columns
    drinks.describe().loc['25%', 'beer_servings']

    drinks.continent.value_counts()['Africa']
    # any Series can be sorted by its values
    drinks.continent.value_counts().sort_values()
    # any Series can also be sorted by its index
    drinks.continent.value_counts().sort_index()

    # concatenate the 'drinks' DataFrame with the 'population' Series (aligns by the index)
    pd.concat([drinks, people], axis=1)

    # calculate the total annual beer servings for each country
    (drinks.beer_servings * people).head()
    #两个数据框对应的值相乘

    # create the 'Sex_male' dummy variable using the 'map' method
    train['Sex_male'] = train.Sex.map({'female':0, 'male':1})
    train.head()

    # create a new 'Location' Series (must use bracket notation to define the Series name)
    ufo['Location'] = ufo.City + ', ' + ufo.State

    ufo.rename(columns={'Colors Reported':'Colors_Reported', 'Shape Reported':'Shape_Reported'}, inplace=True)
    ufo.columns

    # remove multiple columns at once
    ufo.drop(['City', 'State'], axis=1, inplace=True)
    # remove multiple rows at once (axis=0 refers to rows)
    ufo.drop([0, 1], axis=0, inplace=True)

    # create a list in which each element refers to a DataFrame row: True if the row satisfies the condition, False otherwise
    booleans = []
    for length in movies.duration:
        if length >= 200:
            booleans.append(True)
        else:
            booleans.append(False)
    is_long = pd.Series(booleans)
    is_long.head()
    # use bracket notation with the boolean Series to tell the DataFrame which rows to display
    movies[is_long]

    # simplify the steps above: no need to write a for loop to create 'is_long' since pandas will broadcast the comparison
    is_long = movies.duration >= 200
    movies[is_long]

    # or equivalently, write it in one line (no need to create the 'is_long' object)
    movies[movies.duration >= 200]

    处理NA值
    # if 'any' values are missing in a row (considering only 'City' and 'Shape Reported'), then drop that row
    ufo.dropna(subset=['City', 'Shape Reported'], how='any').shape

    # fill in missing values with a specified value
    ufo['Shape Reported'].fillna(value='VARIOUS', inplace=True)

    about index
    # restore the index name, and move the index back to a column
    drinks.index.name = 'country'
    drinks.reset_index(inplace=True)

    # comparison operators work with ordered categories
    df.loc[df.quality > 'good', :]

    get_dummies的用法
    # pass the DataFrame to 'get_dummies' and specify which columns to dummy (it drops the original columns)
    pd.get_dummies(train, columns=['Sex', 'Embarked']).head()


    # only consider a subset of columns when identifying duplicates
    users.duplicated(subset=['age', 'zip_code']).sum()

## pandas一些不熟悉的知识点（二）

    ratings.loc[ratings.movie_id == 1, :].head()
    如果有相同id的列可以直接merge
    movie_ratings = pd.merge(movies, ratings)
    pd.merge(movies, ratings, left_on='m_id', right_on='movie_id').head()

    movies = movies.set_index('m_id')

    A = pd.DataFrame({'color': ['green', 'yellow', 'red'], 'num':[1, 2, 3]})

    2.关于category
    # data types are automatically detected
    drinks.dtypes

    # example of a single aggregation function after a groupby
    drinks.groupby('continent').beer_servings.mean()

    # multiple aggregation functions can be applied simultaneously
    drinks.groupby('continent').beer_servings.agg(['mean', 'min', 'max'])

    About MultiIndex in pandas
    stocks.groupby('Symbol').Close.mean()
    ser = stocks.groupby(['Symbol', 'Date']).Close.mean()
    ser.index #  MultiIndex
    ser.unstack()

    df = stocks.pivot_table(values='Close', index='Symbol', columns='Date')
    df
    此时的df 与ser 是一样的

    Selection from Series with MultiIndex
    ser.loc['AAPL']
    ser.loc['AAPL', '2016-10-03']
    ser.loc[:, '2016-10-03']

    DataFrame with MultiIndex
    stocks.set_index(['Symbol', 'Date'], inplace=True)
    stocks.sort_index(inplace=True)

    Selection from DataFrame with MultiIndex
    stocks.loc[('AAPL', '2016-10-03'), :]
    stocks.loc[('AAPL', '2016-10-03'), 'Close']
    stocks.loc[['AAPL', 'MSFT'], :]
    stocks.loc[(['AAPL', 'MSFT'], '2016-10-03'), :]
    stocks.loc[('AAPL', ['2016-10-03', '2016-10-04']), 'Close']

    # old way to drop rows: specify labels and axis
    ufo.drop([0, 1], axis=0).head()
    ufo.drop([0, 1], axis='index').head()
    # old way to drop columns: specify labels and axis
    ufo.drop(['City', 'State'], axis=1).head()
    ufo.drop(['City', 'State'], axis='columns').head()

    4. rename and reindex now accept "axis" keyword
    New in 0.21.0
    # old way to rename columns: specify columns
    ufo.rename(columns={'City':'CITY', 'State':'STATE'}).head()

    # new way to rename columns: specify mapper and axis
    ufo.rename({'City':'CITY', 'State':'STATE'}, axis='columns').head()
    # note: mapper can be a function
    ufo.rename(str.upper, axis='columns').head()


## 关于pandas 自己写的难记住的

### 
    def dis_type(data):
    if data['distance'] <=2000:
        return '0-2000'
    elif data['distance'] >=2001 and data['distance'] <=4000:
        return '2001-4000'
    else:
        return '>4000'
    #     return data
    hyper.loc[:, "dis_type"] = hyper.apply(dis_type, axis=1)

    pands 取指定的列名及取包含某些字符串的行

    df2=df1.loc[:,MB_E15_MB_sampleid] #MB_E15_MB_sampleid是的list
    MB_E15_MB_sample=df2[df2.gene.str.contains('ENSMUSG00000020893')].T
    #  ENSMUSG00000020893是个基因，可能有很多的转录本